In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold


# data load

In [2]:
titanic_df = pd.read_csv('titanic_train.csv')
titanic_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
titanic_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


# 전처리

## 결측치
- Age : 평균값
- Cabin : N
- Embarked : N

In [5]:
titanic_df['Age'] = titanic_df['Age'].fillna( titanic_df['Age'].mean())
titanic_df['Cabin'] = titanic_df['Cabin'].fillna( 'N')
titanic_df['Embarked'] = titanic_df['Embarked'].fillna( 'N')

titanic_df.isnull().sum().sum()

0

## drop
-PassengerId  
-Name   
-Ticket             

In [6]:

titanic_df = titanic_df.drop(['PassengerId', 'Name', 'Ticket'], axis=1)
titanic_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,male,22.0,1,0,7.2500,N,S
1,1,1,female,38.0,1,0,71.2833,C85,C
2,1,3,female,26.0,0,0,7.9250,N,S
3,1,1,female,35.0,1,0,53.1000,C123,S
4,0,3,male,35.0,0,0,8.0500,N,S


## label encoding - 범주형
Sex,     Cabin,       Embarked

In [8]:
titanic_df['Cabin'] = titanic_df['Cabin'].str[:1]
titanic_df.head(2)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,male,22.0,1,0,7.2500,N,S
1,1,1,female,38.0,1,0,71.2833,C,C


In [10]:
col_names = ['Cabin', 'Sex', 'Embarked']
for col in col_names:
    le = LabelEncoder()
    le.fit(titanic_df[col])
    titanic_df[col] = le.transform(titanic_df[col])

titanic_df.head(2)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,1,22.0,1,0,7.2500,7,3
1,1,1,0,38.0,1,0,71.2833,2,0


## Age, Fare - 수치형
1. 그대로
2. 스케일링
3. 비닝

In [11]:
X_titanic_df = titanic_df.drop('Survived', axis=1)
y_titanic_df = titanic_df['Survived']


# 분할

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X_titanic_df, y_titanic_df, test_size=0.2, random_state=11)

# 학습

In [15]:
dt_clf = DecisionTreeClassifier(random_state=11)
rf_clf = RandomForestClassifier(random_state=11)
lr_clf = LogisticRegression(solver='liblinear')

In [18]:
dt_clf.fit(X_train, y_train )
rf_clf.fit(X_train, y_train)
lr_clf.fit(X_train, y_train)

dt_pred = dt_clf.predict(X_test )
rf_pred = rf_clf.predict(X_test)
lr_pred = lr_clf.predict(X_test)

accuracy_score(y_test, dt_pred), accuracy_score(y_test, rf_pred), accuracy_score(y_test, lr_pred)

(0.7877094972067039, 0.8547486033519553, 0.8659217877094972)

## 교차검증

In [20]:
dt_clf_cv = DecisionTreeClassifier(random_state=11)
rf_clf_cv = RandomForestClassifier(random_state=11)
lr_clf_cv = LogisticRegression(solver='liblinear')

In [21]:
kfold = KFold(n_splits=5) #stratified
scores = []

for train_index, test_index in kfold.split(X_titanic_df) :
    X_train, X_test = X_titanic_df.values[train_index], X_titanic_df.values[test_index]
    y_train, y_test = y_titanic_df.values[train_index], y_titanic_df.values[test_index]

    dt_clf_cv.fit(X_train, y_train )
    dt_pred = dt_clf_cv.predict(X_test )
    scores.append(accuracy_score(y_test, dt_pred))
   
print(scores)
print('평균 정확도 : ', np.mean(scores))


[0.7541899441340782, 0.7808988764044944, 0.7865168539325843, 0.7696629213483146, 0.8202247191011236]
평균 정확도 :  0.782298662984119


In [ ]:
# 함수로 만들기

cross_val_score()

In [ ]:
dt_clf_gcv = DecisionTreeClassifier(random_state=11)
rf_clf_gcv = RandomForestClassifier(random_state=11)
lr_clf_gcv = LogisticRegression(solver='liblinear')

## 파라미터 튜닝 + 교차검증 
GridSearchCV(dt_clf_gcv, param_grid= p)

In [ ]:
GridSearchCV()

# 평가

# 정확도 높이기 - 튜닝